In [1]:
from maverick import Game, GameEvent, GameEventType, ActionType, Player, PlayerAction, PlayerState

In [2]:
class SimpleBot(Player):
    """A simple bot that calls when possible, otherwise checks."""

    def decide_action(self, game, valid_actions, min_raise):
        if ActionType.CALL in valid_actions:
            return PlayerAction(player_id=self.id, action_type=ActionType.CALL)
        elif ActionType.CHECK in valid_actions:
            return PlayerAction(player_id=self.id, action_type=ActionType.CHECK)
        else:
            return PlayerAction(player_id=self.id, action_type=ActionType.FOLD)


class ObservantPlayer(SimpleBot):
    """A player that logs events it observes."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Use object.__setattr__ to bypass Pydantic validation
        object.__setattr__(self, "event_count", 0)

    def on_event(self, event: GameEvent, game: Game) -> None:
        """Track how many events this player observes."""
        # Use object.__setattr__ to bypass Pydantic validation
        object.__setattr__(self, "event_count", self.event_count + 1)

In [3]:
print("=" * 60)
print("Maverick Event System Demo")
print("=" * 60)
print()

# Create a game
game = Game(small_blind=10, big_blind=20, max_hands=1, strict=True)

# Create a global event counter
event_counts = {event_type: 0 for event_type in GameEventType}


def count_events(event: GameEvent, game: Game) -> None:
    """Count each type of event."""
    event_counts[event.type] += 1


def log_major_events(event: GameEvent, game: Game) -> None:
    """Log major game events."""
    if event.type in [
        GameEventType.GAME_STARTED,
        GameEventType.HAND_STARTED,
        GameEventType.HAND_ENDED,
    ]:
        print(f"📢 {event.type.name} (Hand #{event.hand_number}, Street: {event.street.name})")


def log_player_actions(event: GameEvent, game: Game) -> None:
    """Log player actions with details."""
    if event.type == GameEventType.PLAYER_ACTION:
        action_str = f"{event.action.action_type.name}"
        if event.action.amount:
            action_str += f" ({event.action.amount} chips)"
        print(
            f"  🎲 Player {event.player_id}: {action_str} | "
            f"Pot: {game.state.pot} | Current Bet: {game.state.current_bet}"
        )


def log_street_changes(event: GameEvent, game: Game) -> None:
    """Log when streets change."""
    if event.type in [
        GameEventType.DEAL_FLOP,
        GameEventType.DEAL_TURN,
        GameEventType.DEAL_RIVER,
    ]:
        print(f"  🃏 {event.type.name} - Moving to {event.street.name}")


# Register event handlers
game.subscribe(GameEventType.GAME_STARTED, count_events)
game.subscribe(GameEventType.GAME_STARTED, log_major_events)
game.subscribe(GameEventType.GAME_ENDED, count_events)
game.subscribe(GameEventType.GAME_ENDED, log_major_events)
game.subscribe(GameEventType.HAND_STARTED, count_events)
game.subscribe(GameEventType.HAND_STARTED, log_major_events)
game.subscribe(GameEventType.HAND_ENDED, count_events)
game.subscribe(GameEventType.HAND_ENDED, log_major_events)
game.subscribe(GameEventType.PLAYER_ACTION, count_events)
game.subscribe(GameEventType.PLAYER_ACTION, log_player_actions)
game.subscribe(GameEventType.DEAL_HOLE_CARDS, count_events)
game.subscribe(GameEventType.DEAL_HOLE_CARDS, log_player_actions)
game.subscribe(GameEventType.BETTING_ROUND_COMPLETED, count_events)
game.subscribe(GameEventType.BETTING_ROUND_COMPLETED, log_player_actions)
game.subscribe(GameEventType.POST_BLINDS, count_events)
game.subscribe(GameEventType.POST_BLINDS, log_player_actions)
game.subscribe(GameEventType.SHOWDOWN, count_events)
game.subscribe(GameEventType.SHOWDOWN, log_player_actions)
game.subscribe(GameEventType.PLAYER_JOINED, count_events)
game.subscribe(GameEventType.PLAYER_JOINED, log_player_actions)
game.subscribe(GameEventType.PLAYER_LEFT, count_events)
game.subscribe(GameEventType.PLAYER_LEFT, log_player_actions)
game.subscribe(GameEventType.DEAL_FLOP, count_events)
game.subscribe(GameEventType.DEAL_FLOP, log_street_changes)
game.subscribe(GameEventType.DEAL_TURN, count_events)
game.subscribe(GameEventType.DEAL_TURN, log_street_changes)
game.subscribe(GameEventType.DEAL_RIVER, count_events)
game.subscribe(GameEventType.DEAL_RIVER, log_street_changes)


class EventRecorder:
    """Helper class to record events in order."""

    def __init__(self):
        self.events: list[GameEvent] = []

    def record(self, event: GameEvent, game: Game) -> None:
        self.events.append(event)

    def clear(self) -> None:
        self.events = []

    def get_event_types(self) -> list[GameEventType]:
        return [e.type for e in self.events]
    

recorder = EventRecorder()
game.subscribe(GameEventType.GAME_STARTED, recorder.record)

# Add players (one with event hook)
p1 = SimpleBot(id="alice", name="Alice", state=PlayerState(stack=1000))
p2 = ObservantPlayer(id="bob", name="Bob", state=PlayerState(stack=1000))
p3 = SimpleBot(id="charlie", name="Charlie", state=PlayerState(stack=1000))

game.add_player(p1)
game.add_player(p2)
game.add_player(p3)

print(f"Added 3 players: {p1.name}, {p2.name}, {p3.name}")
print()

# Run the game
print("Starting game...")
print()
game.start()

game.remove_player(p1)
game.remove_player(p2)
game.remove_player(p3)

print()
print("=" * 60)
print("Event Summary")
print("=" * 60)
print()

# Print event counts
print("Global Event Counts:")
for event_type, count in sorted(event_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {event_type.name:30s}: {count:3d}")

print()
print(f"Player-Level Event Hooks:")
print(f"  Bob observed {p2.event_count} events through on_event() hook")

print()
print("=" * 60)
print("Demo Complete!")
print("=" * 60)

Maverick Event System Demo

Added 3 players: Alice, Bob, Charlie

Starting game...

📢 GAME_STARTED (Hand #0, Street: PRE_FLOP)
📢 HAND_STARTED (Hand #1, Street: PRE_FLOP)
  🎲 Player alice: CALL | Pot: 50 | Current Bet: 20
  🎲 Player bob: CALL | Pot: 60 | Current Bet: 20
  🎲 Player charlie: CHECK | Pot: 60 | Current Bet: 20
  🃏 DEAL_FLOP - Moving to FLOP
  🎲 Player bob: CHECK | Pot: 60 | Current Bet: 0
  🎲 Player charlie: CHECK | Pot: 60 | Current Bet: 0
  🎲 Player alice: CHECK | Pot: 60 | Current Bet: 0
  🃏 DEAL_TURN - Moving to TURN
  🎲 Player bob: CHECK | Pot: 60 | Current Bet: 0
  🎲 Player charlie: CHECK | Pot: 60 | Current Bet: 0
  🎲 Player alice: CHECK | Pot: 60 | Current Bet: 0
  🃏 DEAL_RIVER - Moving to RIVER
  🎲 Player bob: CHECK | Pot: 60 | Current Bet: 0
  🎲 Player charlie: CHECK | Pot: 60 | Current Bet: 0
  🎲 Player alice: CHECK | Pot: 60 | Current Bet: 0
📢 HAND_ENDED (Hand #1, Street: SHOWDOWN)

Event Summary

Global Event Counts:
  PLAYER_ACTION                 :  12
  BETT